In [1]:
# imports and loading clean data
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

from IPython.display import display, Markdown

from modules.lv_utils import load_households, load_voters

# load the data
households = load_households('data_clean/20180725_fullset_households_district3.csv')
voters = load_voters('data_clean/20180725_fullset_voters_district3.csv')
elections = pd.read_csv('data_clean/20180621_election_data.csv')


Blog/article on manipulating multi_index: https://jakevdp.github.io/PythonDataScienceHandbook/03.05-hierarchical-indexing.html

In [2]:
voters.shape

(13307, 148)

In [3]:
print(voters.columns[0:75])
print(voters.columns[75:])

Index(['Vid', 'Abbr', 'Precinct', 'PrecinctSub', 'Party', 'PartyMain',
       'RegDate', 'PAV', 'RegDateOriginal', 'E6_110816', 'E5_060716',
       'E4_110414', 'E3_060314', 'E2_110612', 'E1_060512', 'District',
       'VScore', 'VScorePos', 'VScorePct', 'BirthYear',
       'OldestInHouseBirthYear', 'IsOldestInHouse', 'havePhone',
       'BirthPlaceState', 'BirthPlaceStateRegion', 'BirthPlaceCountry',
       'BirthPlaceCountryRegion', 'Gender', 'sameMailAddress', 'MailCountry',
       'isApt', 'Zip', 'StreetType', 'EmailProvider', 'E5_060716BT',
       'E1_060512BT', 'Hid', 'cHid', 'E34_nVotesPos', 'E34_nVotes',
       'E34_nVotesPct', 'E56_nVotesPos', 'E56_nVotes', 'E56_nVotesPct',
       'E78_nVotesPos', 'E78_nVotes', 'E78_nVotesPct', 'E12_nVotesPos',
       'E12_nVotes', 'E12_nVotesPct', 'E14_nVotesPos', 'E14_nVotes',
       'E14_nVotesPct', 'E16_nVotesPos', 'E16_nVotes', 'E16_nVotesPct',
       'Eap_nVotesPos', 'Eap_nVotes', 'Eap_nVotesPct', 'Eag_nVotesPos',
       'Eag_nVotes', 'E

In [58]:
#full col list
data_cols = ['Abbr', 'Precinct', 'PrecinctSub', 'Party', 'PartyMain',
       'RegDate', 'PAV', 'RegDateOriginal','VScorePct', 'nVScorePctInHH', 'BirthYear',
       'OldestInHouseBirthYear', 'IsOldestInHouse', 'havePhone',
       'BirthPlaceState', 'BirthPlaceStateRegion', 'BirthPlaceCountry',
       'BirthPlaceCountryRegion', 'Gender', 'sameMailAddress', 'MailCountry',
       'isApt', 'Zip', 'StreetType', 'EmailProvider','nVotersInHH', 'PAVCode', 'nPAVInHH',
       'HasParty', 'nAffInHH', 'isDEM', 'isREP', 'isNPP', 'nDEMInHH',
       'nREPInHH', 'nNPPInHH','BirthYear','nVScorePctInHH',
       'CityArea', 'mostAfflsInHH', 'mixedAfflsInHH',
       'allAffInHH', 'uniformAffInHH']
# cut down list for checking/developing code
#data_cols = ['Party', 'PartyMain','PAV','nVotersInHH', 'PAVCode','HasParty', 'nAffInHH', 'isDEM', 'isREP',]
e_cols = ['E6_110816', 'E5_060716','E4_110414', 'E3_060314', 'E2_110612', 'E1_060512']
gdt_cols = ['E1_GndTth', 'E2_GndTth', 'E3_GndTth', 'E4_GndTth','E5_GndTth', 'E6_GndTth']
nvp_cols = ['E6_nVotesPct','E5_nVotesPct','E4_nVotesPct','E3_nVotesPct','E2_nVotesPct','E1_nVotesPct']
nvphh_cols = ['E1_nVotesPctInHH', 'E2_nVotesPctInHH', 'E3_nVotesPctInHH',
              'E4_nVotesPctInHH', 'E5_nVotesPctInHH', 'E6_nVotesPctInHH']
cv_cols = ['E34_nVotesPct', 'E56_nVotesPct', 'E78_nVotesPct']
cvhh_cols = ['E34_nVotesPctInHH','E56_nVotesPctInHH','E78_nVotesPctInHH']

#turn required series around:
e_cols = e_cols[::-1]
nvp_cols = nvp_cols[::-1]

In [59]:
voters.loc[:5,cv_cols+cvhh_cols]

,E34_nVotesPct,E56_nVotesPct,E78_nVotesPct,E34_nVotesPctInHH,E56_nVotesPctInHH,E78_nVotesPctInHH
vid,,,,,,
0,1.0,1.000000,1.000000,1.0,1.000000,1.000000
1,1.0,0.666667,0.800000,1.0,0.500000,0.600000
2,1.0,0.750000,0.833333,0.8,0.545455,0.705882
3,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000
5,-1.0,-1.000000,-1.000000,0.5,0.250000,0.333333


In [60]:
cols = e_cols+gdt_cols+nvp_cols+nvphh_cols+cv_cols+cvhh_cols
dfw = voters.loc[:,cols+data_cols]
dfw.iloc[:5,5:15]

,E6_110816,E1_GndTth,E2_GndTth,E3_GndTth,E4_GndTth,E5_GndTth,E6_GndTth,E1_nVotesPct,E2_nVotesPct,E3_nVotesPct
vid,,,,,,,,,,
0,A,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,A,-1.0,1.0,0.0,1.0,1.0,1.0,-1.0,1.0,0.0
2,A,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
3,N,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
4,,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [61]:
# relevant question on unstack rather than stack
#https://stackoverflow.com/questions/33057946/unstack-or-pivot-only-some-columns


In [62]:
ele = ['E1','E2','E3','E4','E5','E6','E7','E8']
dfw_data = dfw.drop(columns=cols)
dfw_data.index.rename('vid', inplace=True)

dfw_elecs = dfw[e_cols]
dfw_elecs.index.rename('vid', inplace=True)
dfw_elecs.columns = ele[:6]
dfw_elecs = dfw_elecs.join(pd.DataFrame(np.nan, index=dfw_elecs.index, columns=['E7','E8']))

dfw_gdt = dfw[gdt_cols]
dfw_gdt.index.rename('vid', inplace=True)
dfw_gdt.columns = ele[:6]

dfw_nvp = dfw[nvp_cols]
dfw_nvp.index.rename('vid', inplace=True)
dfw_nvp.columns = ele[1:7]

dfw_nvphh = dfw[nvphh_cols]
dfw_nvphh.index.rename('vid', inplace=True)
dfw_nvphh.columns = ele[1:7]

dfw_cv = dfw[list(np.repeat(cv_cols,2))]
dfw_cv.index.rename('vid', inplace=True)
dfw_cv.columns = ele[2:8]

dfw_cvhh = dfw[list(np.repeat(cvhh_cols,2))]
dfw_cvhh.index.rename('vid', inplace=True)
dfw_cvhh.columns = ele[2:8]

#display(dfw_data)
#display(dfw_elecs)
#display(dfw_gdt)
#display(dfw_nvp)

In [63]:
#pd.DataFrame(dfw_elecs.unstack(level=1).reorder_levels([1,0]))
ustk_e = pd.DataFrame(dfw_elecs.unstack(level=1))
ustk_e.columns = ['vote']
ustk_g = pd.DataFrame(dfw_gdt.unstack(level=1))
ustk_g.columns = ['gndtth']
ustk_l = pd.DataFrame(dfw_nvp.unstack(level=1))
ustk_l.columns = ['lastelec']
ustk_lhh = pd.DataFrame(dfw_nvphh.unstack(level=1))
ustk_lhh.columns = ['lastelechh']
ustk_r = pd.DataFrame(dfw_cv.unstack(level=1))
ustk_r.columns = ['cum']
ustk_rhh = pd.DataFrame(dfw_cvhh.unstack(level=1))
ustk_rhh.columns = ['cumhh']
#display(ustk_e)
#display(ustk_g)

# combining the various reshaped election tables and the person data
data = ustk_e.join([ustk_g, ustk_l, ustk_lhh, ustk_r, ustk_rhh])\
            .join(dfw_data).reorder_levels([1,0]).sort_index(level=0)
data.index.levels[1].name = 'e'

In [64]:
print(data.shape)
data.iloc[:16,:14]

(106456, 49)


vote  gndtth  lastelec  lastelechh       cum  cumhh  Abbr  Precinct  \
vid e                                                                        
0   E1    V     1.0       NaN         NaN       NaN    NaN    82    832400   
    E2    V     1.0       1.0         1.0       NaN    NaN    82    832400   
    E3    V     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E4    V     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E5    A     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E6    A     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E7  NaN     NaN       1.0         1.0  1.000000    1.0    82    832400   
    E8  NaN     NaN       NaN         NaN  1.000000    1.0    82    832400   
1   E1         -1.0       NaN         NaN       NaN    NaN    82    832910   
    E2    V     1.0      -1.0        -1.0       NaN    NaN    82    832910   
    E3    N     0.0       1.0         1.0  1.000000    1.0    82    832910   
    E4    V     1.0       0.0         0.0  1.000000    1.0    82    832910   
    E5    A     1.0       1.0         0.5  0.666667    0.5    82    832910   
    E6    A     1.0       1.0         0.5  0.666667    0.5    82    832910   
    E7  NaN     NaN       1.0         0.8  0.800000    0.6    82    832910   
    E8  NaN     NaN       NaN         NaN  0.800000    0.6    82    832910   

        PrecinctSub Party PartyMain    RegDate PAV RegDateOriginal  
vid e                                                               
0   E1            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E2            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E3            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E4            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E5            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E6            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E7            0   DEM       DEM 1992-10-05   Y      1992-10-05  
    E8            0   DEM       DEM 1992-10-05   Y      1992-10-05  
1   E1            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E2            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E3            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E4            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E5            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E6            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E7            0   DEM       DEM 2012-11-06   Y      2012-10-26  
    E8            0   DEM       DEM 2012-11-06   Y      2012-10-26

In [65]:
elections.index = ele[::-1]
elections.index.name = 'e'
elections

,elections,dates,cycle,etype,president,us_senate_maj,us_repre_maj,ca_governor,ca_lt_govnor,ca_senate_maj,ca_assembly_maj
e,,,,,,,,,,,
E8,E8_110618,2018-11-06,Cong,General,REP,4,42,DEM,DEM,-13,-28
E7,E7_060518,2018-06-05,Cong,Primary,REP,4,42,DEM,DEM,-13,-28
E6,E6_110816,2016-11-08,Pres,General,DEM,10,60,DEM,DEM,-13,-24
E5,E5_060716,2016-06-07,Pres,Primary,DEM,10,58,DEM,DEM,-13,-24
E4,E4_110414,2014-11-04,Cong,General,DEM,-8,34,DEM,DEM,-13,-31
E3,E3_060314,2014-06-03,Cong,Primary,DEM,-8,34,DEM,DEM,-13,-31
E2,E2_110612,2012-11-06,Pres,General,DEM,-3,50,DEM,DEM,-11,-25
E1,E1_060512,2012-06-05,Pres,Primary,DEM,-3,52,DEM,DEM,-11,-25


In [66]:
data = data.join(elections)
data = data.drop(columns=['elections','dates'])  # duplicated information

In [67]:
print(data.shape)
data.iloc[:16,:5+50:]

(106456, 60)


vote  gndtth  lastelec  lastelechh       cum  cumhh  Abbr  Precinct  \
vid e                                                                        
0   E1    V     1.0       NaN         NaN       NaN    NaN    82    832400   
    E2    V     1.0       1.0         1.0       NaN    NaN    82    832400   
    E3    V     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E4    V     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E5    A     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E6    A     1.0       1.0         1.0  1.000000    1.0    82    832400   
    E7  NaN     NaN       1.0         1.0  1.000000    1.0    82    832400   
    E8  NaN     NaN       NaN         NaN  1.000000    1.0    82    832400   
1   E1         -1.0       NaN         NaN       NaN    NaN    82    832910   
    E2    V     1.0      -1.0        -1.0       NaN    NaN    82    832910   
    E3    N     0.0       1.0         1.0  1.000000    1.0    82    832910   
    E4    V     1.0       0.0         0.0  1.000000    1.0    82    832910   
    E5    A     1.0       1.0         0.5  0.666667    0.5    82    832910   
    E6    A     1.0       1.0         0.5  0.666667    0.5    82    832910   
    E7  NaN     NaN       1.0         0.8  0.800000    0.6    82    832910   
    E8  NaN     NaN       NaN         NaN  0.800000    0.6    82    832910   

        PrecinctSub Party      ...       mostAfflsInHH mixedAfflsInHH  \
vid e                          ...                                      
0   E1            0   DEM      ...                 DEM          False   
    E2            0   DEM      ...                 DEM          False   
    E3            0   DEM      ...                 DEM          False   
    E4            0   DEM      ...                 DEM          False   
    E5            0   DEM      ...                 DEM          False   
    E6            0   DEM      ...                 DEM          False   
    E7            0   DEM      ...                 DEM          False   
    E8            0   DEM      ...                 DEM          False   
1   E1            0   DEM      ...                 DEM          False   
    E2            0   DEM      ...                 DEM          False   
    E3            0   DEM      ...                 DEM          False   
    E4            0   DEM      ...                 DEM          False   
    E5            0   DEM      ...                 DEM          False   
    E6            0   DEM      ...                 DEM          False   
    E7            0   DEM      ...                 DEM          False   
    E8            0   DEM      ...                 DEM          False   

       allAffInHH uniformAffInHH  elections       dates  cycle    etype  \
vid e                                                                     
0   E1      False          False  E1_060512  2012-06-05   Pres  Primary   
    E2      False          False  E2_110612  2012-11-06   Pres  General   
    E3      False          False  E3_060314  2014-06-03   Cong  Primary   
    E4      False          False  E4_110414  2014-11-04   Cong  General   
    E5      False          False  E5_060716  2016-06-07   Pres  Primary   
    E6      False          False  E6_110816  2016-11-08   Pres  General   
    E7      False          False  E7_060518  2018-06-05   Cong  Primary   
    E8      False          False  E8_110618  2018-11-06   Cong  General   
1   E1      False          False  E1_060512  2012-06-05   Pres  Primary   
    E2      False          False  E2_110612  2012-11-06   Pres  General   
    E3      False          False  E3_060314  2014-06-03   Cong  Primary   
    E4      False          False  E4_110414  2014-11-04   Cong  General   
    E5      False          False  E5_060716  2016-06-07   Pres  Primary   
    E6      False          False  E6_110816  2016-11-08   Pres  General   
    E7      False          False  E7_060518  2018-06-05   Cong  Primary   
    E8      

In [68]:
data = data.reset_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106456 entries, 0 to 106455
Data columns (total 60 columns):
vid                        106456 non-null int64
e                          106456 non-null object
vote                       79842 non-null object
gndtth                     79842 non-null float64
lastelec                   79842 non-null float64
lastelechh                 79842 non-null float64
cum                        79842 non-null float64
cumhh                      79842 non-null float64
Abbr                       106456 non-null int64
Precinct                   106456 non-null int64
PrecinctSub                106456 non-null int64
Party                      106456 non-null category
PartyMain                  106456 non-null object
RegDate                    106456 non-null datetime64[ns]
PAV                        106456 non-null category
RegDateOriginal            106456 non-null datetime64[ns]
VScorePct                  106456 non-null float64
nVScorePctInHH         

In [74]:
obj = data.select_dtypes(include=object)
obj.describe()

,e,vote,PartyMain,BirthPlaceStateRegion,BirthPlaceCountryRegion,EmailProvider,CityArea,mostAfflsInHH,cycle,etype,president,ca_governor,ca_lt_govnor
count,106456,79842,106456,106456,106456,106456,106456,106456,106456,106456,106456,106456,106456
unique,8,4,5,5,7,202,3,18,2,2,2,1,1
top,E2,N,DEM,UNK,USA,UNK,Centerville,DEM,Cong,Primary,DEM,DEM,DEM
freq,13307,30329,53072,50912,56592,72072,73496,60544,53228,53228,79842,106456,106456


# Trying first Model

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
X = df.drop(columns=['vid','vote','e','gndtth'])
y = df.gndtth

In [36]:
df = data[data.e.isin(['E3','E4','E5','E6'])]

In [37]:
model = RandomForestClassifier()
model = model.fit(X,y)

ValueError: could not convert string to float: 'DEM'